In [1]:
%aiida
import ase
import aiida
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm import Group, JobCalculation,  WorkCalculation

In [2]:
def retrieve_alluncalculated_structures(structure_group, calculation_group=None):
    Group.get_from_string(structure_group) #check if structure group exists 
    
    sqb = QueryBuilder()
    sqb.append(Group, filters={'name': structure_group}, tag='g')
    sqb.append(StructureData, project='id', tag='s', member_of='g')
    sqb.append(WorkCalculation, tag='job', output_of='s')
    
    filters = {}
    if calculation_group:
        Group.get_from_string(calculation_group) #check if calculation_group exists
        filters={'name':calculation_group}
    sqb.append(Group, group_of='job', filters=filters)

    ids_dealt_with = [_ for _, in sqb.distinct().all()] or [-1] # prevent empty list if none calculated!

    # # Now the main query:
    qb = QueryBuilder()
    qb.append(Group, filters={'name':structure_group}, tag='g')
    qb.append(StructureData, project='*', tag='s', member_of='g',
        filters={'id':{'!in':ids_dealt_with}}) # This will filter out calculated, !in for not in
    
    return qb.all() #or maybe just qb?

In [4]:
# calc_g = Group.get_or_create(name="test_calc_group")[0]

# run_calc = load_node(13100)
# calc_g.add_nodes(run_calc)

In [5]:
STRUCTURE_GROUP="twoatom_tenrandom_1"
CALCULATION_GROUP="twoatom_tenrandom_1_calc1"

In [6]:
try:
    len(retrieve_alluncalculated_structures(STRUCTURE_GROUP, calculation_group="bb"))
except Exception, e:
    print e

No user-defined group with name 'bb' found.


In [7]:
len(retrieve_alluncalculated_structures(STRUCTURE_GROUP, calculation_group=CALCULATION_GROUP))

0

In [8]:
len(retrieve_alluncalculated_structures(STRUCTURE_GROUP, calculation_group=STRUCTURE_GROUP))

10

4893